<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Iris-Classification-with-Keras" data-toc-modified-id="Iris-Classification-with-Keras-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Iris Classification with Keras</a></span></li><li><span><a href="#Load-libraries-and-data" data-toc-modified-id="Load-libraries-and-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load libraries and data</a></span></li><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper functions</a></span></li><li><span><a href="#Inspect-and-visualize-the-data" data-toc-modified-id="Inspect-and-visualize-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Inspect and visualize the data</a></span><ul class="toc-item"><li><span><a href="#Descriptive-statistics" data-toc-modified-id="Descriptive-statistics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Descriptive statistics</a></span></li><li><span><a href="#Visualizations" data-toc-modified-id="Visualizations-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Visualizations</a></span></li></ul></li><li><span><a href="#Model-the-data" data-toc-modified-id="Model-the-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model the data</a></span><ul class="toc-item"><li><span><a href="#Create-validation-data-set" data-toc-modified-id="Create-validation-data-set-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Create validation data set</a></span></li><li><span><a href="#Build-the-model" data-toc-modified-id="Build-the-model-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Build the model</a></span><ul class="toc-item"><li><span><a href="#Initial-pass" data-toc-modified-id="Initial-pass-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Initial pass</a></span></li><li><span><a href="#Hyperparameter-tuning" data-toc-modified-id="Hyperparameter-tuning-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Hyperparameter tuning</a></span><ul class="toc-item"><li><span><a href="#Predictions" data-toc-modified-id="Predictions-5.2.2.1"><span class="toc-item-num">5.2.2.1&nbsp;&nbsp;</span>Predictions</a></span></li></ul></li></ul></li></ul></li></ul></div>

<h1>Iris Classification with Keras</h1>

<img style="float: left; margin-right: 15px;" src="images/iris.jpg" />

# Purpose

The purpose of this write-up is to build upon the [first](https://nbviewer.jupyter.org/github/nrasch/Portfolio/blob/master/Machine-Learning/Python/04-Classic-Datasets/Model-01.ipynb) write-up involving the Iris dataset.  

Goals include:
* Build a predictive regression model via neural networks
* Perform hyperparameter tuning on the neural network
* Make predictions with the training model and examine accuracy

Dataset source:  [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)



# Load libraries and data

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [73]:
# Load libraries
import os

import numpy as np

from pandas import read_csv
from pandas.plotting import scatter_matrix
from pandas import set_option
from pandas import DataFrame

from matplotlib import pyplot

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

In [5]:
dataFile = os.path.join(".", "datasets", "iris.data.csv")
data = read_csv(dataFile, header = 0)

# Helper functions

In [38]:
def makeRange(start, stop, step, multi, dec):
    vals = []
    for i in range(start, stop, step):
        vals.append(np.round(multi * i, decimals = dec))
        
    return vals

# Inspect and visualize the data

Please the [first Iris data's write-up](https://nbviewer.jupyter.org/github/nrasch/Portfolio/blob/master/Machine-Learning/Python/04-Classic-Datasets/Model-01.ipynb#Inspect-and-visualize-the-data) details on this topic.

# Model the data

## Create validation data set

In [17]:
# Seperate X and Y values
x = data.values[:, 0:4]
y = data.values[:, 4]

print("x.shape = ", x.shape)
print("y.shape = ", y.shape)

DataFrame(y).groupby([0]).size()


x.shape =  (150, 4)
y.shape =  (150,)


0
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
dtype: int64

Convert the Y values to one hot encodings for the neural network.

In [18]:
_tmp = LabelEncoder().fit(y).transform(y)
yHot = np_utils.to_categorical(_tmp)
yHot.shape

(150, 3)

Create the validation set utilizing the one hot encodings.

In [19]:
# Split out validation set -- 80/20 split
seed = 10
valSize = 0.2

xTrain, xVal, yTrain, yVal = train_test_split(x, yHot, test_size = valSize, random_state = seed)

print("--------")
print("xTrain.shape = ", xTrain.shape)
print("yTrain.shape = ", yTrain.shape)
print("xVal.shape = ", xVal.shape)
print("yVal.shape = ", yVal.shape)

--------
xTrain.shape =  (120, 4)
yTrain.shape =  (120, 3)
xVal.shape =  (30, 4)
yVal.shape =  (30, 3)


## Build the model

### Initial pass

To use Keras with Scikit-Learn we'll utilize these handy [wrappers](https://keras.io/scikit-learn-api/).  So first thing we need to do is write a function to build our model that we can pass to the `build_fn` parameter.

In [20]:
# To do:  Edit function to accept params and dynamicly set everything up...
def buildModel():
    model = Sequential()
    model.add(Dense(8, input_dim = 4, activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model


Now we can write the rest of the code:

In [56]:
# Define vars and init
folds = 10
seed = 10

np.random.seed(seed)

model = KerasClassifier(build_fn = buildModel, epochs = 200, batch_size = 5, verbose = 0)
kFold = KFold(n_splits = folds, random_state = seed)
results = cross_val_score(model, xTrain, yTrain, cv = kFold)

print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Accuracy: 97.50% (5.34%)


Next well see how the model performs on the test data:

In [34]:
# Create the model and train it
model = buildModel()
model.fit(xTrain, yTrain, epochs = 200, batch_size = 5, verbose = 0)

# Run the model against the test data
scores = model.evaluate(xVal, yVal, verbose=0)
print("Model error: %.2f%%" % (100-scores[1]*100))

# Examine the predictions for the classes
preds = model.predict_classes(xVal)
trueVals = np.argmax(yVal, axis = 1)

print("Accuracy score: %.2f%%" % (accuracy_score(trueVals, preds) * 100), "\n")
print("Confusion Matrix\n", confusion_matrix(trueVals, preds), "\n")
print("Classification Report\n", classification_report(trueVals, preds), "\n")

print("--------")
print("Actual class predictions vs. true values:")
print("Y-hat : ", preds)
print("Y     : ", trueVals)

Model error: 3.33%
Accuracy score: 96.67% 

Confusion Matrix
 [[10  0  0]
 [ 0 12  1]
 [ 0  0  7]] 

Classification Report
              precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       1.00      0.92      0.96        13
          2       0.88      1.00      0.93         7

avg / total       0.97      0.97      0.97        30
 

--------
Actual class predictions vs. true values:
Y-hat :  [1 2 0 1 0 1 2 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]
Y     :  [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]


### Hyperparameter tuning

We'll borrow a function we wrote in a previous write-up for [Sonar, Mines vs. Rocks](./Model-03.ipynb):

In [76]:
def tuneModel(modelName, modelObj, params, returnModel = False, showSummary = True):
    # Init vars and params
    featureResults = {}
    featureFolds = 10
    featureSeed = 10

    # Use accuracy since this is a classification problem
    score = 'accuracy'

    # Create a Pandas DF to hold all our spiffy results
    featureDF = DataFrame(columns = ['Model', 'Accuracy', 'Best Params'])

    # Create feature union
    features = []
    features.append(('Scaler', StandardScaler()))
    featureUnion = FeatureUnion(features)

    # Search for the best combination of parameters
    featureResults = GridSearchCV(
        Pipeline(
            steps = [
                ('FeatureUnion', featureUnion),
                (modelName, modelObj)
        ]),
        param_grid = params,
        scoring = score,
        cv = KFold(n_splits = featureFolds, random_state = featureSeed)      
    ).fit(xTrain, np.argmax(yTrain, axis=1))

    featureDF.loc[len(featureDF)] = list([
        modelName, 
        featureResults.best_score_,
        featureResults.best_params_,
    ])

    if showSummary:
        set_option('display.max_colwidth', -1)
        display(featureDF)
    
    if returnModel:
        return featureResults

Note the following:  https://github.com/keras-team/keras/issues/9331

Because a categorical transformation was applied to the `y` values above the following error was occuring:

```
ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets.
```

To resolve this the categorical transformation in the `tuneModel` function above needs to be reversed:

```python
).fit(xTrain, np.argmax(yTrain, axis=1))
```

Also note that the data inspection indicated the iris didn't need to have the `StandardScaler` applied.  However, for the sake of completeness that feature will be implemented in the `tuneModel` code.

In [77]:
modelName = "irisTuned"
modelObj =  KerasClassifier(build_fn = buildModel, verbose = 0)
params = {
    'irisTuned__epochs' : makeRange(100, 200, 50, 1, 1),
    'irisTuned__batch_size' : makeRange(1, 3, 1, 1, 1),
}

tuneModel(modelName, modelObj, params)

,Model,Accuracy,Best Params
0,irisTuned,0.96,"{'irisTuned__batch_size': 1, 'irisTuned__epochs': 150}"


#### Predictions

In [79]:
# Create the model and train it
model = buildModel()
model.fit(xTrain, yTrain, epochs = 150, batch_size = 1, verbose = 0)

# Run the model against the test data
scores = model.evaluate(xVal, yVal, verbose=0)
print("Model error: %.2f%%" % (100-scores[1]*100))

# Examine the predictions for the classes
preds = model.predict_classes(xVal)
trueVals = np.argmax(yVal, axis = 1)

print("Accuracy score: %.2f%%" % (accuracy_score(trueVals, preds) * 100), "\n")
print("Confusion Matrix\n", confusion_matrix(trueVals, preds), "\n")
print("Classification Report\n", classification_report(trueVals, preds), "\n")

print("--------")
print("Actual class predictions vs. true values:")
print("Y-hat : ", preds)
print("Y     : ", trueVals)

Model error: 3.33%
Accuracy score: 96.67% 

Confusion Matrix
 [[10  0  0]
 [ 0 12  1]
 [ 0  0  7]] 

Classification Report
              precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       1.00      0.92      0.96        13
          2       0.88      1.00      0.93         7

avg / total       0.97      0.97      0.97        30
 

--------
Actual class predictions vs. true values:
Y-hat :  [1 2 0 1 0 1 2 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]
Y     :  [1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2]


Not surprisingly we get pretty much the same results....